# Musicians - Hard

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)


 ········


In [2]:
band = pd.read_sql_table('band', engine)
composer = pd.read_sql_table('composer', engine)
composition = pd.read_sql_table('composition', engine)
concert = pd.read_sql_table('concert', engine)
has_composed = pd.read_sql_table('has_composed', engine)
musician = pd.read_sql_table('musician', engine)
performance = pd.read_sql_table('performance', engine)
performer = pd.read_sql_table('performer', engine)
place = pd.read_sql_table('place', engine)
plays_in = pd.read_sql_table('plays_in', engine)

## 11.
**List the name and town of birth of any performer born in the same city as James First.**

In [3]:
jf = (musician.loc[musician['m_name']=='James First']
      .merge(place, left_on='born_in', right_on='place_no')
      [['born_in', 'place_town']])
(musician.loc[musician['m_name'] != 'James First']
 .merge(jf, on='born_in')[['m_name', 'place_town']].sort_values('m_name'))

,m_name,place_town
1,Alan Fluff,London
2,Andy Jones,London
0,Theo Mengel,London


## 12.
**Create a list showing for EVERY musician born in Britain the number of compositions and the number of instruments played.**

In [4]:
t = (musician.merge(place.loc[place['place_country'].isin(['England', 'Scotland'])],
                    left_on='born_in', right_on='place_no', how='left')
     .merge(composer, left_on='m_no', right_on='comp_is', how='left')
     .merge(has_composed, left_on='comp_no', right_on='cmpr_no', how='left')
     .merge(composition, left_on='cmpn_no', right_on='c_no', how='left')
     .groupby(['m_no', 'm_name'])
     .agg(n_composition=pd.NamedAgg(column='c_no', aggfunc='count'))
     .reset_index())
p = (musician.merge(performer, left_on='m_no', right_on='perf_is', how='left')
     [['m_no', 'instrument']].drop_duplicates()
     .groupby('m_no').agg(n_instrument=pd.NamedAgg(column='instrument', aggfunc='count'))
     .reset_index())
t.merge(p, on='m_no').sort_values('m_name')
# t

,m_no,m_name,n_composition,n_instrument
13,14,Alan Fluff,0,2
18,19,Andy Jones,1,0
11,12,Davis Heavan,0,3
17,18,Elsie James,0,3
0,1,Fred Bloggs,2,0
16,17,Freda Miles,2,0
3,4,Harriet Smithson,0,2
7,8,Harry Forte,2,3
2,3,Helen Smyth,1,1
4,5,James First,4,1


## 13.
**Give the band name, conductor and contact of the bands performing at the most recent concert in the Royal Albert Hall.**

In [5]:
rah = (concert.loc[concert['concert_venue']=='Royal Albert Hall']
       .sort_values('con_date', ascending=False).iloc[:1])
(band.merge(musician.rename(columns={'m_name': 'contact'}), 
            left_on='band_contact', right_on='m_no')
 .merge(performance, left_on='band_no', right_on='gave')
 .merge(musician.rename(columns={'m_name': 'conductor'}),
        left_on='conducted_by', right_on='m_no')
 .merge(rah, left_on='performed_in', right_on='concert_no')
 [['band_name', 'contact', 'conductor']])

,band_name,contact,conductor
0,Somebody Loves this,Theo Mengel,Alan Fluff


## 14.
**Give a list of musicians associated with Glasgow. Include the name of the musician and the nature of the association - one or more of 'LIVES_IN', 'BORN_IN', 'PERFORMED_IN' AND 'IN_BAND_IN'.**

In [6]:
t1 = (musician.merge(place, left_on='born_in', right_on='place_no')
      [['m_name', 'place_town']]
      .assign(assoc='BORN_IN'))
t2 = (musician.merge(place, left_on='living_in', right_on='place_no')
      [['m_name', 'place_town']]
      .assign(assoc='LIVES_IN'))
t3 = (musician.merge(performer, left_on='m_no', right_on='perf_is')
      .merge(plays_in, left_on='perf_no', right_on='player')
      .merge(performance, left_on='band_id', right_on='gave')
      .merge(concert, left_on='performed_in', right_on='concert_no')
      .merge(place, left_on='concert_in', right_on='place_no')
      [['m_name', 'place_town']]
      .assign(assoc='PERFORMED_IN'))
t4 = (musician.merge(plays_in, left_on='m_no', right_on='player')
      .merge(band, left_on='band_id', right_on='band_no')
      .merge(place, left_on='band_home', right_on='place_no')
      [['m_name', 'place_town']]
      .assign(assoc='IN_BAND_IN'))
t = pd.concat([t1, t2, t3, t4])
t.loc[t['place_town']=='Glasgow', ['m_name', 'assoc']].drop_duplicates()

,m_name,assoc
17,Lovely Time,BORN_IN
18,Louise Simpson,BORN_IN
19,Steven Chaytors,BORN_IN
8,Harriet Smithson,LIVES_IN
9,Jeff Dawn,LIVES_IN
10,Andy Jones,LIVES_IN
11,Louise Simpson,LIVES_IN
12,James Steeple,LIVES_IN
13,Jeff Dawn,IN_BAND_IN
14,Davis Heavan,IN_BAND_IN


## 15.
**Jeff Dawn plays in a band with someone who plays in a band with Sue Little. Who is it and what are the bands?**

In [7]:
sue_band = (musician.loc[musician['m_name']=='Sue Little']
            .merge(performer, left_on='m_no', right_on='perf_is')
            .merge(plays_in, left_on='perf_no', right_on='player')
            .merge(band, left_on='band_id', right_on='band_no')
            [['band_id', 'band_name']])
sue_coplayers = (musician
                 .merge(performer, left_on='m_no', right_on='perf_is')
                 .merge(plays_in, left_on='perf_no', right_on='player')
                 .merge(sue_band, on='band_id'))
jeff_band = (musician.loc[musician['m_name']=='Jeff Dawn']
             .merge(performer, left_on='m_no', right_on='perf_is')
             .merge(plays_in, left_on='perf_no', right_on='player')
             .merge(band, left_on='band_id', right_on='band_no')
             [['band_id', 'band_name']])
jeff_coplayers = (musician
                 .merge(performer, left_on='m_no', right_on='perf_is')
                 .merge(plays_in, left_on='perf_no', right_on='player')
                 .merge(jeff_band, on='band_id'))
(sue_coplayers.merge(jeff_coplayers, on='m_no')
 [['m_name_x', 'band_name_x', 'band_name_y']]
 .rename(columns={'m_name_x': 'm_name', 'band_name_x': 'Sue\'s band', 
                  'band_name_y': 'Jeff\'s band'}))

,m_name,Sue's band,Jeff's band
0,John Smith,BBSO,AASO
